In [2]:
import sys
try:
    sys.path.insert(0, "/usr/lib/python3.7/site-packages")
except FileNotFoundError:
    pass

import pandas as pd
import numpy as nm
import sys
import os
import fasttext
import bert_dp
import tensorflow

from deeppavlov import build_model, configs
os.environ["KERAS_BACKEND"] = "tensorflow"

In [3]:
tensorflow.__version__

'1.13.1'

In [4]:
CONFIG_PATH = "/home/fomius2000/DeepPavlov/deeppavlov/configs/classifiers/rusentiment_bert.json"

model = build_model(CONFIG_PATH, download=False)

print(model(["Я очень люблю устанавливать модули на питон"]))

[nltk_data] Downloading package punkt to /home/fomius2000/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/fomius2000/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package perluniprops to
[nltk_data]     /home/fomius2000/nltk_data...
[nltk_data]   Package perluniprops is already up-to-date!
[nltk_data] Downloading package nonbreaking_prefixes to
[nltk_data]     /home/fomius2000/nltk_data...
[nltk_data]   Package nonbreaking_prefixes is already up-to-date!
2019-11-14 13:42:31.537 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/fomius2000/.deeppavlov/models/classifiers/rusentiment_bert_v0/classes.dict]
Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


2019-11-14 13:42:57.89 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/fomius2000/.deeppavlov/models/classifiers/rusentiment_bert_v0/model]


INFO:tensorflow:Restoring parameters from /home/fomius2000/.deeppavlov/models/classifiers/rusentiment_bert_v0/model
['neutral']


In [5]:
sents = ["Я в восторге от этой модели", 
         "Он ненавидит", 
         "Она абсолютно не участвовала в политике.",
         "Он абсолютно не отвечает за свои действия.",
         "ЧеловекX бегом спустился в долину.",
         "ЧеловекX абсолютно выбился из сил.",
         "Окровавленный ЧеловекX замертво упал на землю."]
df = pd.DataFrame(columns=["sentence", "polarity"])
df["sentence"] = sents
df["polarity"] = model(sents)
df

,sentence,polarity
0,Я в восторге от этой модели,positive
1,Он ненавидит,negative
2,Она абсолютно не участвовала в политике.,neutral
3,Он абсолютно не отвечает за свои действия.,neutral
4,ЧеловекX бегом спустился в долину.,neutral
5,ЧеловекX абсолютно выбился из сил.,negative
6,Окровавленный ЧеловекX замертво упал на землю.,skip


In [6]:
class nice_out:
    def __init__(self, L):
        self.L = L
        self.counter = 0
        self.per = 0
    def update(self):
        if(self.per == (self.counter*1000)//self.L):
            print("\r" + str(self.per/10) + "%" + " " + "#"*(self.per//20) + "_"*(50 - self.per//20), end = "")
            self.per += 1
        self.counter += 1
    def clear(self):
        print("\r" + " "*100, end = "\r")

In [7]:
all_sent = pd.read_csv("Dataset_persX.csv")

L = len(all_sent)
all_sent = all_sent[:(L//1000)*1000]
print(all_sent.head(20))

cnt = all_sent.groupby("root")
cnt.head(20)

nums = all_sent.groupby(["root"]).size().reset_index(name='counts')
nums = nums.sort_values(["counts"], ascending=[False])
print("\n", len(nums))
nums.head(30)

#L /= 1000
#chk = nice_out(len(all_sent))
#all_sent["polarity"] = "0"
#for i in range(0, len(all_sent), 1000):
#    chk.update()
#    batch = all_sent[i: i + 1000]
#    all_sent[i: i + 1000]["polarity"] = model(all_sent[i : i + 1000]["sentence"].tolist())
#chk.clean()

#all_sent["polarity"] = all_sent["sentence"].apply(model)
#all_sent.head()

                                             sentence           root
0        Встревоженный ЧеловекX пошел навстречу мэру.          пойти
1           Бывший ЧеловекX превратился в настоятеля.   превращаться
2            Старый ЧеловекX прохаживался возле дома.  прохаживаться
3     Лопоухий ЧеловекX по-прежнему был в прострации.     прострация
4               Высокий ЧеловекX вернулся с граблями;      вернуться
5              Сморщенный ЧеловекX сидел за столиком.         сидеть
6                Обессилевший ЧеловекX уснул на полу.         уснуть
7   Старый Абрахам безмятежно спал за столиком адм...          спать
8          ЧеловекX абсолютно не разбираюсь в музыке.    разбираться
9         ЧеловекX абсолютно забыла о своем обещании.       забывать
10  ЧеловекX абсолютно не оправдались в действител...  оправдываться
11          ЧеловекX абсолютно не считался с фактами.      считаться
12                 ЧеловекX абсолютно выбился из сил.     выбиваться
13       ЧеловекX абсолютно не дум

,root,counts
7637,смотреть,19364
767,взглядывать,7333
1460,выходить,6121
715,вернуться,5309
5368,подходить,5040
5444,пойти,4649
5107,поворачиваться,4294
7978,стоять,4274
2733,идти,3846
7479,сидеть,3610


In [8]:
same_root = []
for it, row in nums[100:300].iterrows():
    same_root += [all_sent[all_sent["root"] == row["root"]]]


In [9]:
for i in range(0, 200):
    print("\r      " + str(i))
    tmp_lst = model(same_root[i]["sentence"].tolist())
    same_root[i]["polarity"] = tmp_lst

      0


/home/fomius2000/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


      1
      2
      3
      4
      5
      6
      7
      8
      9
      10
      11
      12
      13
      14
      15
      16
      17
      18
      19
      20
      21
      22
      23
      24
      25
      26
      27
      28
      29
      30
      31
      32
      33
      34
      35
      36
      37
      38
      39
      40
      41
      42
      43
      44
      45
      46
      47
      48
      49
      50
      51
      52
      53
      54
      55
      56
      57
      58
      59
      60
      61
      62
      63
      64
      65
      66
      67
      68
      69
      70
      71
      72
      73
      74
      75
      76
      77
      78
      79
      80
      81
      82
      83
      84
      85
      86
      87
      88
      89
      90
      91
      92
      93
      94
      95
      96
      97
      98
      99
      100
      101
      102
      103
      104
      105
      106
      107
      108
      109
      110
      11

In [16]:
result = same_root[0]
for i in range(1, 200):
    result = result.append(same_root[i], ignore_index=True)
print(result.shape)

(65332, 3)


In [18]:
result = result[result["polarity"] != "neutral"]
print(result.shape)
result.head()
result.to_csv("Not_neutral.csv")

(6564, 3)


In [ ]:
#print(len(tmp_lst), len(tmp_tab))
same_root[99]["polarity"] = tmp_lst

In [ ]:
print(same_root[99][same_root[99]["polarity"] != "neutral"])

In [ ]:
pol = same_root[99]["sentence"].apply(model)
#print(same_root[99].shape)